In [2]:
from tensorflow.keras.models import load_model
import tensorflow as tf
import os

In [6]:
# save to regular tflite
model = load_model('mnist_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# save the model
with open('mnist_model.tflite', 'wb') as f:
  f.write(tflite_model)

tflite_size = os.path.getsize("mnist_model.tflite")
h5_size = os.path.getsize('mnist_model.h5')
# print(tflite_size, h5_size)

2023-10-25 16:21:55.526884: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1600]
	 [[{{node inputs}}]]
2023-10-25 16:21:55.531694: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,128]
	 [[{{node inputs}}]]
2023-10-25 16:21:55.604461: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1600]
	 [[{{node inputs}}]]
2023-10-25 16:

INFO:tensorflow:Assets written to: /var/folders/qm/p5vffjb56gvb80sz8bwjxy780000gn/T/tmptv8s06vw/assets


INFO:tensorflow:Assets written to: /var/folders/qm/p5vffjb56gvb80sz8bwjxy780000gn/T/tmptv8s06vw/assets
2023-10-25 16:21:56.235439: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-10-25 16:21:56.235452: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-10-25 16:21:56.235945: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/qm/p5vffjb56gvb80sz8bwjxy780000gn/T/tmptv8s06vw
2023-10-25 16:21:56.237124: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-10-25 16:21:56.237138: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/qm/p5vffjb56gvb80sz8bwjxy780000gn/T/tmptv8s06vw
2023-10-25 16:21:56.241229: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-10-25 16:21:56.302883: I tensorflow/cc/saved_model/loader.cc:215] Running initialization

In [7]:
# save to optimized tflite
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# save the model
with open('mnist_model_optimized.tflite', 'wb') as f:
  f.write(tflite_model)

tflite_opt_size = os.path.getsize('mnist_model.h5')
print(tflite_size, h5_size, tflite_opt_size)

2023-10-25 16:21:58.700619: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1600]
	 [[{{node inputs}}]]
2023-10-25 16:21:58.705544: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,128]
	 [[{{node inputs}}]]
2023-10-25 16:21:58.773020: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1600]
	 [[{{node inputs}}]]
2023-10-25 16:

INFO:tensorflow:Assets written to: /var/folders/qm/p5vffjb56gvb80sz8bwjxy780000gn/T/tmpfg8tcx1v/assets


INFO:tensorflow:Assets written to: /var/folders/qm/p5vffjb56gvb80sz8bwjxy780000gn/T/tmpfg8tcx1v/assets


903620 2748704 2748704


2023-10-25 16:21:59.194770: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-10-25 16:21:59.194783: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-10-25 16:21:59.194921: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/qm/p5vffjb56gvb80sz8bwjxy780000gn/T/tmpfg8tcx1v
2023-10-25 16:21:59.195998: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-10-25 16:21:59.196003: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/qm/p5vffjb56gvb80sz8bwjxy780000gn/T/tmpfg8tcx1v
2023-10-25 16:21:59.199722: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-10-25 16:21:59.236435: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /var/folders/qm/p5vffjb56gvb80sz8bwjxy780000gn/T/tmpfg8tcx1v
2023-10-

In [8]:
# call tflite model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.image import rgb_to_grayscale
from PIL import Image, ImageOps
import tensorflow as tf
from time import time
import numpy as np

tflite_path = 'mnist_model.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
input_tensor_index = input_details[0]['index']

output_details = interpreter.get_output_details()
# print(output_details)

output = interpreter.tensor(interpreter.get_output_details()[0]["index"])
print(output)


<function Interpreter.tensor.<locals>.<lambda> at 0x159ef22a0>


INFO: Applying 1 TensorFlow Lite delegate(s) lazily.
VERBOSE: Replacing 8 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 1 partitions for the whole graph.
INFO: Successfully applied the default TensorFlow Lite delegate indexed at 0.
 *NOTE*: because a delegate has been applied, the precision of computations should be unchanged, but the exact output tensor values may have changed. If such output values are checked in your code, like in your tests etc., please consider increasing error tolerance for the check.


In [11]:
# call tflite model
img = load_img('number/3.png', target_size=(28,28))

img = ImageOps.invert(img)
img = img_to_array(img)
img = rgb_to_grayscale(img)
img = img/255.0
img = np.expand_dims(img, axis=0)

interpreter.set_tensor(input_tensor_index, img)
interpreter.invoke()

digit = np.argmax(output()[0])
print(digit)

7
